In [ ]:
# default_exp transform

In [ ]:
# export
import torch

In [ ]:
# export
def deg2rad(angles):
    if torch.__version__ >= '1.6':
        return torch.deg2rad(angles)

    M_PI_180 = torch.tensor(0.017453292519943295769236907684886127134428718885417, dtype=torch.float64)
    return torch.mul(angles, M_PI_180.to(angles.device))

In [ ]:
# export
def axis_angle_rotation(axis: str, angle, mode='normal'):
    """
    Return the rotation matrices for one of the rotations about an axis
    of which Euler angles describe, for each value of the angle given.

    Args:
        axis: Axis label "X" or "Y or "Z".
        angle: any shape tensor of Euler angles in radians

    Returns:
        Rotation matrices as tensor of shape (..., 3, 3).
    """

    cos = torch.cos(angle).to(angle.device)
    sin = torch.sin(angle).to(angle.device)
    one = torch.ones_like(angle).to(angle.device)
    zero = torch.zeros_like(angle).to(angle.device)
    
    if mode == 'normal':
        ff = 1
    elif mode == '3ddfa':
        ff = -1

    if axis == "X":
        R_flat = (one, zero, zero, 
                  zero, cos, -sin*ff,
                  zero, sin*ff, cos)
    if axis == "Y":
        R_flat = (cos, zero, sin*ff,
                  zero, one, zero,
                  -sin*ff, zero, cos)
    if axis == "Z":
        R_flat = (cos, -sin*ff, zero,
                  sin*ff, cos, zero,
                  zero, zero, one)

    return torch.stack(R_flat, -1).reshape(angle.shape + (3, 3))

def angle2matrix(angles, mode='normal'):
    angles = angles.double()
    if mode == 'normal':
        matrices = map(axis_angle_rotation, "ZYX", torch.unbind(deg2rad(angles), -1)[::-1], [mode]*3)
    elif mode =='3ddfa':
        matrices = map(axis_angle_rotation, "XYZ", torch.unbind(angles, -1), [mode]*3)
    return functools.reduce(torch.matmul, matrices).float()

In [ ]:
# export
def similarity_transform(verts, s, R, t3d):
    tfm_verts = s[..., None] * verts.matmul(R.transpose(-1, -2)) + t3d.unsqueeze(-2)
    return tfm_verts

def transform(vertices, s, angles, t3d, mode='normal'):
    R = angle2matrix(angles, mode)
    return similarity_transform(vertices.float(), s, R, t3d)

In [ ]:
# export
# TODO
def rotate(vertices, angles):
    ''' rotate vertices. 
    X_new = R.dot(X). X: 3 x 1   
    Args:
        vertices: [nver, 3]. 
        rx, ry, rz: degree angles
        rx: pitch. positive for looking down 
        ry: yaw. positive for looking left
        rz: roll. positive for tilting head right
    Returns:
        rotated vertices: [nver, 3]
    '''
    R = angle2matrix(angles)
    rotated_vertices = vertices.dot(R.T)

    return rotated_vertices

In [ ]:
# export
# TODO
def matrix2angle(R):
    ''' get three Euler angles from Rotation Matrix
    Args:
        R: (3,3). rotation matrix
    Returns:
        x: pitch
        y: yaw
        z: roll
    '''
    assert(isRotationMatrix)
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    # rx, ry, rz = np.rad2deg(x), np.rad2deg(y), np.rad2deg(z)
    rx, ry, rz = x*180/np.pi, y*180/np.pi, z*180/np.pi
    return rx, ry, rz

In [ ]:
# export
def denormal_mesh(verts, mean=None, std=None, pose=None):
    if std is not None:
        verts *= std
    if mean is not None:
        verts += mean
    if pose is not None:
        s = pose[6]/1000/0.0012
        angles = pose[:3]
        t = pose[3:6]*225
        verts = transform(verts, s, angles, t, '3ddfa')
    return verts

In [ ]:
from nbdev.export import *

notebook2script()

Converted index.ipynb.
Converted io.ipynb.
Converted transform.ipynb.
Converted utils.ipynb.
